In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from inspect import signature
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
import copy
from scipy.interpolate import CubicSpline
from scipy.stats import gaussian_kde
import tqdm
import functools
from scipy import integrate


In [ ]:
class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

class MLP(nn.Module):
    def __init__(self,
                 in_dim = 1,
                 out_dim = 1,
                 widths = [64, 128, 128, 64]):
        super(MLP, self).__init__()
        
        widths = [in_dim] + list(widths) + [out_dim]
        self.n_layers = len(widths) - 1
        self.layers = nn.ModuleList([nn.Linear(widths[i], widths[i+1]) for i in range(self.n_layers)])
        #self.activation = Swish()#nn.ReLU()
        self.activation = nn.LeakyReLU(inplace=True)
    def forward(self, x):
        for i in range(self.n_layers - 1):
            x = self.layers[i](x)
            x = self.activation(x)
        x = self.layers[-1](x)
        return x

#----------------------------

def train(model, optimizer, scheduler, training_loader, val_loader, epochs = 10, loss =  nn.MSELoss(), freq_print = 1, tag = ""):

    best_model_testing_error = 1000

    for epoch in range(epochs):
        train_mse = 0.0
        for step, (input_batch, output_batch) in enumerate(training_loader):
            optimizer.zero_grad()
            output_pred_batch = model(input_batch)
            loss_f = loss(output_pred_batch, output_batch)
            loss_f.backward()
            optimizer.step()
            train_mse += loss_f.item()
        train_mse /= len(training_loader)

        scheduler.step()

        with torch.no_grad():
            model.eval()
            test_relative_l2 = 0.0
            for step, (input_batch, output_batch) in enumerate(val_loader):
                output_pred_batch = model(input_batch)
                #loss_f = (torch.mean((abs(output_pred_batch - output_batch))) / torch.mean(abs(output_batch))) * 100
                #loss_f = torch.mean(100* torch.norm(output_pred_batch - output_batch, p=2, dim = [1])/torch.norm(output_batch, p=2,dim = [1]))
                loss_f = torch.mean(torch.norm(output_pred_batch - output_batch, p=2, dim = [1]))

                test_relative_l2 += loss_f.item()
            test_relative_l2 /= len(val_loader)


        if test_relative_l2 < best_model_testing_error:
            best_model_testing_error = test_relative_l2
            best_model = copy.deepcopy(model)
            torch.save(best_model, f"saved_models/model_1d_{tag}.pkl")

        if epoch % freq_print == 0: print("######### Epoch:", epoch, " ######### Train Loss:", train_mse, " ######### Relative L1 Test Norm:", test_relative_l2)

    return best_model
    

In [ ]:
%pip install xitorch

In [ ]:
def f_sine(x):
    #return x
    return np.where(x < 0, np.sin(np.pi*x/2), np.sin(25*np.pi * x))

def get_dataset(N, batch_size=64):
    X = np.random.uniform(-1,1,N)
    Y = f_sine(X).reshape(-1,1)
    X = X.reshape(-1,1)
    return torch.tensor(X).to("cuda").type(torch.float32), torch.tensor(Y).to("cuda").type(torch.float32)


In [ ]:
X = np.linspace(-1,1,1000)
plt.scatter(X, f_sine(X), s = 12)
plt.plot(X, f_sine(X))
plt.grid()

#plt.savefig("Figures/1d_diffusion/sines/true_fn.pdf", dpi = 400)
#plt.savefig("Figures/1d_diffusion/sines/true_fn.png", dpi = 400)

In [ ]:
batch_size = 128
N_train = 5000
N_val = 1024
N_test = 1024


inp_train, out_train = get_dataset(N_train, batch_size=batch_size)
inp_val, out_val = get_dataset(N_val, batch_size=batch_size)
inp_test, out_test = get_dataset(N_test, batch_size=batch_size)

#-------------

dataset_train = TensorDataset(inp_train, out_train)
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
dataset_val = TensorDataset(inp_val, out_val)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False)

learning_rate = 0.001
weight_decay = 0
step_size = 100
gamma = 0.8
epochs = 500

tag = "tmp"

model = MLP(widths = [64, 64]).to("cuda")
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

#model = train(model, optimizer, scheduler, dataloader_train, dataloader_val, epochs = epochs, freq_print=50, tag = tag)



In [ ]:

inp_test, out_test = get_dataset(5000)
plt.plot(X, f_sine(X), label = "True")

Y_pred = model(inp_test)[:,0]
plt.scatter(inp_test[:,0].detach().cpu().numpy(), Y_pred.detach().cpu().numpy(), color = "orange", label = "Prediction (MLP)", s = 15)

plt.legend(fontsize = 12)
plt.grid(True)

#plt.savefig("Figures/1d_diffusion/sines/predicted_reg_fn.pdf", dpi = 400)
#plt.savefig("Figures/1d_diffusion/sines/predicted_reg_fn.png", dpi = 400)

In [ ]:
def marginal_prob_std_1(t, sigma, device = "cuda"):
    """
    Args:
    t: A vector of time steps.
    sigma: The $\sigma$ in our SDE.

    Returns:
    The standard deviation.
    """
    t = torch.tensor(t, device=device)
    return torch.sqrt((sigma**(2 * t) - 1.) / 2. / np.log(sigma))

def diffusion_coeff_1(t, sigma, device = "cuda"):
    """Compute the diffusion coefficient of our SDE.

    Args:
    t: A vector of time steps.
    sigma: The $\sigma$ in our SDE.

    Returns:
    The vector of diffusion coefficients.
    """
    return torch.tensor(sigma**t, device=device)

def marginal_prob_std_2(t, sigma_min, sigma_max, device = "cuda"):
    """
    Args:
    t: A vector of time steps.
    sigma: The $\sigma$ in our SDE.

    Returns:
    The standard deviation.
    """
    t = torch.tensor(t, device=device)
    #print(torch.sqrt(sigma_min * torch.pow(sigma_max/sigma_min, t)))
    return sigma_min * torch.pow(sigma_max/sigma_min, t)

def diffusion_coeff_2(t, sigma_min, sigma_max, device = "cuda"):
    """Compute the diffusion coefficient of our SDE.

    Args:
    t: A vector of time steps.
    sigma: The $\sigma$ in our SDE.

    Returns:
    The vector of diffusion coefficients.
    """
    return torch.tensor(sigma_min * torch.pow(sigma_max/sigma_min, t) * np.sqrt(2*np.log(sigma_max/sigma_min)), device=device)

#----------------------

class MLPDiffusion(nn.Module):
    def __init__(self,
                 in_dim = 1,
                 out_dim = 1,
                 cond_dim = 0,
                 widths = [64, 128, 128, 64],
                 marginal_prob_std = None):
        super(MLPDiffusion, self).__init__()
        
        widths = [in_dim + 1 + cond_dim] + list(widths) + [out_dim]
        self.n_layers = len(widths) - 1
        self.layers = nn.ModuleList([nn.Linear(widths[i], widths[i+1]) for i in range(self.n_layers)])
        self.activation = nn.SiLU()

        self.marginal_prob_std = marginal_prob_std


    def forward(self, 
                x,
                x_cond,
                diffusion_time):
        
        sigma = self.marginal_prob_std(diffusion_time)
        #sigma = torch.log(sigma) + 1/2.

        if sigma.dim() < 1:
            sigma = sigma.expand(x.shape[0])
        
        if x_cond is not None:
            x = torch.cat((x, x_cond, torch.log(sigma)[:, None]), dim=1)
        else:
            x = torch.cat((x, torch.log(sigma)[:, None]), dim=1)
        
        
        for i in range(self.n_layers - 1):
            x = self.layers[i](x)
            x = self.activation(x)
        x = self.layers[-1](x)
        return x/sigma[:,None]

In [ ]:
def loss_fn(model, 
            x, 
            x_cond,
            marginal_prob_std = None, 
            eps=1e-6,
            is_train = True):

        if is_train:
                random_t = torch.rand(x.shape[0], device=x.device) * (1. - eps) + eps    
                z = torch.randn_like(x, device = x.device)    
                std = marginal_prob_std(random_t) 
                #score = model(x + z * std[:, None], x_cond, random_t)
                #loss = torch.mean(torch.mean((score * std[:, None] + z)**2, dim=(1)))
                
                denoised = model(x + z * std[:, None], x_cond, random_t)

                x_squared = torch.square(x)
                denoise_squared = torch.square(denoised)
                loss = torch.mean(torch.square(denoised - x))
                return loss
        else:
                err_val = 0
                for level in range(8):
                        t_min = level/8.0
                        t_max = (level + 1)/8.0
                        random_t = (torch.rand(x.shape[0], device=x.device) * (1. - eps) + eps )*(t_max-t_min)+t_min
                        
                        z = torch.randn_like(x)
                        std = marginal_prob_std(random_t) 

                        denoised = model(x + z * std[:, None], x_cond, random_t)
                        loss = torch.mean(torch.square(denoised - x))
                        err_val = err_val + loss
                return err_val/8.0

In [ ]:
from diffusion.model import EMA

def Euler_Maruyama_sampler(model,
                           marginal_prob_std,
                           diffusion_coeff,
                           condition,
                           batch_size=64,
                           num_steps=128,
                           device='cuda',
                           dimension = (2,),
                           eps=1e-4):
  """Generate samples from score-based models with the Euler-Maruyama solver.

  Args:
    model: A PyTorch model that represents the time-dependent score-based model.
    marginal_prob_std: A function that gives the standard deviation of
      the perturbation kernel.
    diffusion_coeff: A function that gives the diffusion coefficient of the SDE.
    batch_size: The number of samplers to generate by calling this function once.
    num_steps: The number of sampling steps.
      Equivalent to the number of discretized time steps.
    device: 'cuda' for running on GPUs, and 'cpu' for running on CPUs.
    eps: The smallest time step for numerical stability.

  Returns:
    Samples.
  """

  device = next(model.parameters()).device

  t = torch.ones(batch_size, device=device)
  init_x = torch.randn(batch_size, dimension[0], device=device) * marginal_prob_std(t).to(device)[:, None]
  time_steps = torch.linspace(1., eps, num_steps, device=device)
  step_size = time_steps[0] - time_steps[1]
  x = init_x
  with torch.no_grad():
    for time_step in tqdm.tqdm(time_steps):
        batch_time_step = torch.ones(batch_size, device=device) * time_step
        g = diffusion_coeff(batch_time_step).to(device)
        #score = model(x, condition, batch_time_step)
        denoised = model(x, condition, batch_time_step)
        std = marginal_prob_std(batch_time_step)
        score = (denoised - x)/std[:, None]**2
        mean_x = x + (g**2)[:, None] * score * step_size
        x = mean_x + torch.sqrt(step_size) * g[:, None] * torch.randn_like(x)
        
  return mean_x

In [ ]:
def ode_sampler(score_model,
                marginal_prob_std,
                diffusion_coeff,
                condition = None,
                batch_size=64,
                atol=1e-5,
                rtol=1e-5,
                device='cuda',
                z=None,
                dimension = (2,),
                eps=1e-3):
  """Generate samples from score-based models with black-box ODE solvers.

  Args:
    score_model: A PyTorch model that represents the time-dependent score-based model.
    marginal_prob_std: A function that returns the standard deviation
      of the perturbation kernel.
    diffusion_coeff: A function that returns the diffusion coefficient of the SDE.
    batch_size: The number of samplers to generate by calling this function once.
    atol: Tolerance of absolute errors.
    rtol: Tolerance of relative errors.
    device: 'cuda' for running on GPUs, and 'cpu' for running on CPUs.
    z: The latent code that governs the final sample. If None, we start from p_1;
      otherwise, we start from the given z.
    eps: The smallest time step for numerical stability.
  """

  t = torch.ones(batch_size, device=device)
  # Create the latent code
  if z is None:
    init_x = torch.randn(batch_size, dimension[0], device=device) * marginal_prob_std(t)[:, None]
  else:
    init_x = z

  shape = init_x.shape

  def score_eval_wrapper(sample, time_steps):
    """A wrapper of the score-based model for use by the ODE solver."""
    score_model.train()
    sample = torch.tensor(sample, device=device, dtype=torch.float32).reshape(shape)
    time_steps = torch.tensor(time_steps, device=device, dtype=torch.float32).reshape((sample.shape[0], ))
    denoised = score_model(sample, None, time_steps)
    std = marginal_prob_std(time_steps)
    
    #score = (denoised - sample)/std[:, None]**2
    
    return ((denoised - sample)/std[:, None]**2).detach().cpu().numpy().reshape((-1))

  def ode_func(t, x):
    """The ODE function for use by the ODE solver."""
    time_steps = np.ones((shape[0],)) * t
    g = diffusion_coeff(torch.tensor(t)).cpu().numpy()
    return  -0.5 * (g**2) * score_eval_wrapper(x, time_steps)

  # Run the black-box ODE solver.
  res = integrate.solve_ivp(ode_func, (1., eps), init_x.reshape(-1).cpu().numpy(), rtol=rtol, atol=atol, method='RK45')
  print(f"Number of function evaluations: {res.nfev}")
  x = torch.tensor(res.y[:, -1], device=device).reshape(shape)

  return x


In [ ]:
from torch.func import vmap, jvp

def prior_likelihood(z, sigma):  
    """The likelihood of a Gaussian distribution with mean zero and
      standard deviation sigma."""
    shape = z.shape
    N = np.prod(shape[1:])
    return -N / 2. * np.log(2*np.pi*sigma**2) - np.sum(z**2, axis=(1,)) / (2 * sigma**2)


def ode_likelihood(model,
                   x,
                   condition,
                   marginal_prob_std,
                   diffusion_coeff,
                   t_batch = None,
                   batch_size=64,
                   device='cuda',
                   eps = 1e-6,
                   rtol = 1e-6,
                   atol = 1e-6,
                   epsilon = None):

    shape = x.shape
    B,C = shape[0], np.prod(shape[1:])
    shape_epsilon = (batch_size,)+shape[1:] 
    #epsilon = torch.randn(shape_epsilon, device = device).type(torch.float32) #Shape (batchsize, other)
    #epsilon = torch.sqrt(torch.prod(torch.tensor(shape[1:], device=device))) * epsilon / torch.norm(epsilon, dim=1, keepdim=True)

    """def divergence_eval(fn, x, epsilon_all):
        '''
        Args:
            fn: Python function mapping tensor input x to tensor output (output.shape==x.shape)
            x: torch.Tensor of shape (B,other), x is the point at which divergence is estimated
            epsilon_all: torch.Tensor of shape (sample_size,other), samples for Hutchinson estimator

        Returns:
            div_f_estimates: torch.Tensor of shape (B, sample_size), contains values
                            epsilon_all[j]^T . Jfn(x[i]) . epsilon_all[j]
        '''
        # Define the JVP computation
        
        def e_Jf_e(eps):
            _, jvp_result = jvp(f_e, (x, eps), (eps.repeat(B,1), torch.zeros_like(eps)))
            return jvp_result

        #def f_e(x, eps):
        #    return torch.sum(fn(x).reshape(B,C) * eps.view(1,-1), dim=-1)

        #x = torch.tensor(x,  device=device, dtype=torch.float32)
        #x.requires_grad_(True)
        #epsilon_all.requires_grad_(True)
        
        with torch.enable_grad():

            '''
            print("X.requires_grad:", x.requires_grad)  
            print("Epsilon shape:", epsilon_all.shape)
            print("Function output shape:", fn(x).shape)
            print("fn(x) requires grad:", fn(x).requires_grad)
            '''
            # define relevant functions and gradients
            f_e = lambda x, eps: torch.sum(fn(x).reshape(B,C) * eps.view(1,-1), dim=-1) # shape (B,)
            e_Jf_e = lambda eps: jvp(f_e, (x, eps), (eps.repeat(B,1), torch.zeros_like(eps)))[1]
            e_Jf_e_multisample = vmap(e_Jf_e, out_dims=1) # allow additional "batch-dimension" for eps (this is sample_size)

            # get the estimates
            #print(epsilon_all)
            div_f_estimates = e_Jf_e_multisample(epsilon_all) # x: (B, other), epsilon_all: (sample_size, other)
            #print(div_f_estimates)
            return div_f_estimates.reshape(-1,)"""

    def divergence_eval(fn, x, epsilon):
        """Compute the divergence of the score-based model with Skilling-Hutchinson."""
        
        #with torch.enable_grad():

        #res = torch.zeros((B,batch_size), device = device).type(torch.float32)
        #for i in range(batch_size):
        epsilon.requires_grad_(True)
        x.requires_grad_(True)
        f_e = lambda x, eps: torch.sum(fn(x).reshape(B,C) * eps.view(1,-1), dim=-1) # shape (B,)
        #e_Jf_e = lambda eps: torch.autograd.functional.jvp(f_e, (x, eps), (eps.repeat(B,1), torch.zeros_like(eps)))[1]
        #res[:,i] =  e_Jf_e(epsilon[i])

        def e_Jf_e(eps):
            _, jvp_result = jvp(f_e, (x, eps), (eps.repeat(B,1), torch.zeros_like(eps)))
            return jvp_result

        e_Jf_e_multisample = vmap(e_Jf_e, out_dims=1) # allow additional "batch-dimension" for eps (this is sample_size)
        #res = torch.mean(res, dim = 1)
        res = e_Jf_e_multisample(epsilon)
        return res.mean(dim=1)

    def ode_func(t, x):
        """The ODE function for the black-box solver."""
        time_steps = np.ones((shape[0],)) * t   
        sample = x[:B*C]
        logp = x[B*C:]
        g = diffusion_coeff(torch.tensor(t))
        
        sample = torch.as_tensor(sample, device=device, dtype=torch.float32).reshape(shape)  # Convert to tensor
        
        time_steps = time_steps.reshape((sample.shape[0],))
        

        time_steps = torch.tensor(time_steps, device=device, dtype=torch.float32)
        sample = torch.tensor(sample, device=device, dtype=torch.float32)
        std = marginal_prob_std(time_steps)[:, None]

        #sample.requires_grad_(True)
        #epsilon.requires_grad_(True)
        #epsilon = torch.randn((batch_size, 2), device = device).type(torch.float32)
        #epsilon = torch.sqrt(torch.prod(torch.tensor(2, device=device))) * epsilon / torch.norm(epsilon, dim=1, keepdim=True)
        model.train()
        fn = lambda x: (model(x, condition, time_steps.reshape((B,))) - x.reshape(shape)) / std**2


        logp_grad = -0.5 * g**2 * divergence_eval(fn, sample, epsilon)
        sample_grad = -0.5 * g**2 * fn(sample)
        
        return np.concatenate([sample_grad.reshape(-1).detach().cpu().numpy(), logp_grad.reshape(-1).detach().cpu().numpy()], axis=0)

    init = np.concatenate([x.cpu().numpy().reshape((-1,)), np.zeros((shape[0],))], axis=0)

    # Black-box ODE solver
    res = integrate.solve_ivp(ode_func, (eps, 1.), init, rtol=rtol, atol=atol, method='RK45')
    
    zp = res.y[:,-1]

    z, delta_logp = zp[:B*C], zp[B*C:]
    z = z.reshape((B,C))

    sigma_max = marginal_prob_std(torch.tensor(1.0, device=device)).detach().cpu().numpy()
    prior_logp = prior_likelihood(z, sigma_max)

    N = np.prod(shape[1:])
    
    return z, prior_logp/N, delta_logp/N


In [ ]:
from torch.func import vmap, jvp
from xitorch import integrate as integrate_torch

def prior_likelihood_torch(z, sigma):  
    """The likelihood of a Gaussian distribution with mean zero and
      standard deviation sigma."""
    shape = z.shape
    N = torch.prod(torch.tensor(shape[1:], device=device))
    dim = [i for i in range(1, len(shape))]
    return -N / 2. * torch.log(2*np.pi*sigma**2) - torch.sum(z**2, dim=dim) / (2 * sigma**2)


def ode_likelihood_torch(model,
                        x,
                        condition,
                        marginal_prob_std,
                        diffusion_coeff,
                        t_batch = None,
                        batch_size=64,
                        device='cuda',
                        eps = 1e-6,
                        rtol = 1e-6,
                        atol = 1e-6,
                        epsilon = None):

    shape = x.shape
    B,C = shape[0], torch.tensor(shape[1:], device=device)
    shape_epsilon = (batch_size,)+shape[1:] 

    def divergence_eval(fn, x, epsilon):
        """Compute the divergence of the score-based model with Skilling-Hutchinson."""

        epsilon.requires_grad_(True)
        x.requires_grad_(True)
        f_e = lambda x, eps: torch.sum(fn(x).reshape(B,C) * eps.view(1,-1), dim=-1) # shape (B,)

        def e_Jf_e(eps):
            _, jvp_result = jvp(f_e, (x, eps), (eps.repeat(B,1), torch.zeros_like(eps)))
            return jvp_result

        e_Jf_e_multisample = vmap(e_Jf_e, out_dims=1) # allow additional "batch-dimension" for eps (this is sample_size)
        res = e_Jf_e_multisample(epsilon)
        return res.mean(dim=1)

    def ode_func(t, x):
        """The ODE function for the black-box solver."""
        time_steps = torch.ones((shape[0],), device = device) * t   
        sample = x[:B*C]
        logp = x[B*C:]
        g = diffusion_coeff(torch.tensor(t, device=device))
        
        sample = sample.reshape(shape)  # Convert to tensor
        time_steps = time_steps.reshape((sample.shape[0],))

        std = marginal_prob_std(time_steps)[(slice(None),) + (None,) * (len(shape) - 1)]
        model.train()
        fn = lambda x: (model(x, condition, time_steps.reshape((B,))) - x.reshape(shape)) / std**2

        logp_grad = -0.5 * g**2 * divergence_eval(fn, sample, epsilon)
        sample_grad = -0.5 * g**2 * fn(sample)
        
        return torch.cat([sample_grad.reshape(-1), logp_grad.reshape(-1)], axis = 0)

    '''
    init = np.concatenate([x.cpu().numpy().reshape((-1,)), np.zeros((shape[0],))], axis=0)
    # Black-box ODE solver
    res = integrate.solve_ivp(ode_func, (eps, 1.), init, rtol=rtol, atol=atol, method='RK45')
    zp = res.y[:,-1]
    '''
    init = torch.cat([x.view(-1), torch.zeros(shape[0], device=device)])
    #res = odeint(ode_func, init, torch.tensor([eps, 1.0], device=device), rtol=rtol, atol=atol, method='heun2') #heun2 #rk4
    
    res = integrate_torch.solve_ivp(fcn = ode_func, y0=init, ts = torch.tensor([eps, 1.0], device=device), rtol=rtol, atol=atol, method="rk38")
    zp = res[-1]
    
    z, delta_logp = zp[:B*C], zp[B*C:]
    z = z.reshape((B,C))

    sigma_max = marginal_prob_std(torch.tensor(1.0, device=device))
    prior_logp = prior_likelihood_torch(z, sigma_max)

    N = torch.prod(torch.tensor(shape[1:], device=device))
    
    return z, prior_logp/N, delta_logp/N


In [ ]:
learning_rate = 0.001
step_size = 50
gamma = 0.95
epochs = 1
device = "cuda"

tqdm_epoch = tqdm.trange(epochs)

save_epochs = [100, 1000, 2000, 5000, 9999]

which_type = "x&y"
if which_type == "x&y":
  cond_dim = 0
  in_dim = 2
else:
  cond_dim = 1
  in_dim = 1

sigma = 100.

#marginal_prob_std_fn = functools.partial(marginal_prob_std_1, sigma=sigma, device = device)
#diffusion_coeff_fn = functools.partial(diffusion_coeff_1, sigma=sigma, device = device)

marginal_prob_std_fn = functools.partial(marginal_prob_std_2, sigma_min = 0.001, sigma_max=sigma, device = device)
diffusion_coeff_fn = functools.partial(diffusion_coeff_2, sigma_min = 0.001, sigma_max=sigma, device = device)

model_diff = MLPDiffusion(widths = [1024, 1024, 1024], marginal_prob_std = marginal_prob_std_fn, cond_dim=cond_dim, in_dim=in_dim, out_dim=in_dim).to("cuda")
optimizer = AdamW(model_diff.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

best_avg_loss = 1000.
best_model_diff = None

for epoch in tqdm_epoch:
    avg_loss = 0.
    num_items = 0
    avg_loss_val = 0.

    model_diff.train()

    for input_batch, output_batch in dataloader_train:
      input_batch = input_batch.to(device)
      output_batch = output_batch.to(device)

      if which_type == "yx":
        loss = loss_fn(model_diff, output_batch, input_batch, marginal_prob_std_fn)
        condition = input_batch
      elif which_type == "x&y":
        loss = loss_fn(model_diff, torch.cat((input_batch, output_batch), axis = 1), None, marginal_prob_std_fn)
        condition = None
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      avg_loss += loss.item() * output_batch.shape[0]
      num_items += output_batch.shape[0]
    avg_loss /= num_items

    scheduler.step()

    model_diff.eval()
    
    num_items = 0
    with torch.no_grad():
        for input_batch, output_batch in dataloader_val:
          input_batch = input_batch.to(device)
          output_batch = output_batch.to(device)

          if which_type == "yx":
            loss = loss_fn(model_diff, output_batch, input_batch, marginal_prob_std_fn, is_train = False)
          elif which_type == "x&y":
            loss = loss_fn(model_diff, torch.cat((input_batch, output_batch), axis = 1), None, marginal_prob_std_fn, is_train = False)
          avg_loss_val += loss.item() * output_batch.shape[0]
          num_items += output_batch.shape[0]
    avg_loss_val/= num_items

    
    if avg_loss_val<best_avg_loss:
      best_avg_loss = avg_loss_val    
      best_model_diff = copy.deepcopy(model_diff)
    

    if epoch in save_epochs:
      torch.save(best_model_diff.state_dict(), f"saved_models/diff_model1d_ep_{epoch}.pth")

    if epoch%10 == 0:
      tqdm_epoch.set_description('Train: {:.5f} Val: {:.5f}'.format(avg_loss, avg_loss_val))
    

In [ ]:
sigma = 100.
cond_dim = 0
in_dim = 2
marginal_prob_std_fn = functools.partial(marginal_prob_std_2, sigma_min = 0.001, sigma_max=sigma, device = device)
diffusion_coeff_fn = functools.partial(diffusion_coeff_2, sigma_min = 0.001, sigma_max=sigma, device = device)

path =  "/path_to_model/"
device = "cuda"
model_best = MLPDiffusion(widths = [1024, 1024, 1024], marginal_prob_std = marginal_prob_std_fn, cond_dim=cond_dim, in_dim=in_dim, out_dim=in_dim).to("cuda")
model_best.load_state_dict(torch.load(path, map_location=device, weights_only=True))
model_best.eval()


samples_sde = Euler_Maruyama_sampler(model_best,
                                marginal_prob_std_fn,
                                diffusion_coeff_fn,
                                None,
                                batch_size=2048,
                                num_steps=512,
                                device='cuda',
                                dimension = (2,),
                                eps=1e-10)

samples_ode = ode_sampler(model_best,
                marginal_prob_std_fn,
                diffusion_coeff_fn,
                condition = None,
                batch_size=1024,
                atol=1e-6,
                rtol=1e-6,
                device='cuda',
                z=None,
                dimension = (2,),
                eps=1e-10)

plt.scatter(samples_ode[:,0].detach().cpu().numpy(),samples_ode[:,1].detach().cpu().numpy(), color = "blue", s = 16, label = "Sampled (ODE)")
X = np.linspace(-1,1.05,1000)
plt.plot(X, f_sine(X), "--", color = "red", label = "Ground Truth")
plt.grid()

plt.xlim([-1,1])
plt.ylim([-1.05,1.05])
plt.legend(fontsize=14)

plt.xlabel("x",fontsize=14)
plt.ylabel("f(x)",fontsize=14)

#plt.savefig("Figures/1d_diffusion/sines/gen_samples_ODE_fn.pdf", dpi = 400)
#plt.savefig("Figures/1d_diffusion/sines/gen_samples_ODE_fn.png", dpi = 400)
plt.show()

plt.scatter(samples_sde[:,0].detach().cpu().numpy(),samples_sde[:,1].detach().cpu().numpy(), color = "green", s = 16, label = "Sampled (SDE)")
X = np.linspace(-1,1.05,1000)
plt.plot(X, f_sine(X), "--", color = "red", label = "Ground Truth")
plt.grid()

plt.xlim([-1,1])
plt.ylim([-1.05,1.05])
plt.legend(fontsize=14)

plt.xlabel("x",fontsize=14)
plt.ylabel("f(x)",fontsize=14)

plt.savefig("Figures/1d_diffusion/sines/gen_samples_SDE_fn.pdf", dpi = 400)
plt.savefig("Figures/1d_diffusion/sines/gen_samples_SDE_fn.png", dpi = 400)
plt.show()


In [ ]:
plt.scatter(coordinates[:,0].detach().cpu().numpy(), coordinates[:,1].detach().cpu().numpy(), s = 0.25, label = "Target Points")
plt.plot([0,0],[-l- 0.05,l+ 0.05], color = "blue")
plt.plot([-l- 0.05,l+ 0.05],[0,0], color = "blue")
plt.xlim([-l - 0.05, l + 0.05])
plt.ylim([-l - 0.05, l + 0.05])
plt.legend(fontsize = 12)

plt.xlabel("x coordinate", fontsize = 12)
plt.ylabel("y coordinate", fontsize = 12)
    
#plt.savefig("Figures/1d_diffusion/sines/target_points_likelihood.pdf", dpi = 400)
#plt.savefig("Figures/1d_diffusion/sines/target_points_likelihood.png", dpi = 400)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

S = 128  # Example resolution
step_size = (1/(S))

# Generate 1D grid points
l = 1.0
x = np.arange(-l, l, step_size*(l))
y = np.arange(-l, l, step_size*(l))

# Create 2D grid
xx, yy = np.meshgrid(x, y, indexing='ij')
print(xx.shape)
# Stack coordinates
coordinates = np.column_stack([xx.ravel(), yy.ravel()])
coordinates = torch.tensor(coordinates).to("cuda").type(torch.float32)

path = "/path_to_model/"
device = "cuda"
model_best = MLPDiffusion(widths = [1024, 1024, 1024], marginal_prob_std = marginal_prob_std_fn, cond_dim=cond_dim, in_dim=in_dim, out_dim=in_dim).to("cuda")
model_best.load_state_dict(torch.load(path, map_location=device, weights_only=True))


# Create a dataset
dataset = TensorDataset(coordinates)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=256, shuffle=False)

# Iterate over the DataLoader

batch_size = 256
epsilon = torch.randn((batch_size, 2), device = device).type(torch.float32)
epsilon = torch.sqrt(torch.prod(torch.tensor(2, device=device))) * epsilon / torch.norm(epsilon, dim=1, keepdim=True)
for i, batch in enumerate(dataloader):
    print(i, len(dataloader))
    x_, prior_, delta_ = ode_likelihood_torch(model= model_best,
                                            x = batch[0],
                                            condition = None,
                                            marginal_prob_std = marginal_prob_std_fn,
                                            diffusion_coeff = diffusion_coeff_fn,
                                            batch_size = batch_size,
                                            device='cuda',
                                            eps = 1e-8,
                                            rtol = 1e-2,
                                            atol = 1e-2,
                                            epsilon = epsilon)
                        
    if i == 0:
        x, prior, delta = x_.detach().cpu().numpy(), prior_.detach().cpu().numpy(), delta_.detach().cpu().numpy()
    else:
        x = np.concatenate((x, x_.detach().cpu().numpy()))
        prior = np.concatenate((prior, prior_.detach().cpu().numpy()))
        delta = np.concatenate((delta, delta_.detach().cpu().numpy()))


In [ ]:
print(xx.shape)
xx =xx.reshape((2*S, 2*S))
mask_neg = np.logical_and(xx < 0, np.abs(f_sine(xx)-yy)<1000)
mask_neg = mask_neg.reshape(-1)
mask_pos = xx > 0 
mask_pos = mask_pos.reshape(-1)

X = x[:,0]/sigma
Y = x[:,1]/sigma

for i,mask in enumerate([mask_pos, mask_neg]):
    Y_plt = Y[mask]
    X_plt = X[mask]
    
    bins = 100
    plt.hist2d(X_plt,Y_plt, bins = bins, density= True, cmap = "gist_ncar")
    plt.colorbar()

    plt.xlabel("x coordinate of the prior", fontsize = 12)
    plt.ylabel("y coordinate of the prior", fontsize = 12)
    
    '''
    if i ==0:
        plt.savefig("Figures/1d_diffusion/sines/prior_hist_positive_x.pdf", dpi = 400)
        plt.savefig("Figures/1d_diffusion/sines/prior_hist_positive_x.png", dpi = 400)
    elif i ==1:
        plt.savefig("Figures/1d_diffusion/sines/prior_hist_negative_x.pdf", dpi = 400)
        plt.savefig("Figures/1d_diffusion/sines/prior_hist_negative_x.png", dpi = 400)
    '''
    plt.show()


Y_pos = Y[mask_pos]
X_pos = X[mask_pos]
Y_neg = Y[mask_neg]
X_neg = X[mask_neg]

mask_neg = np.logical_and(np.abs(X_neg)<np.max(X_pos), np.abs(Y_neg)<np.max(Y_pos))
Y_neg = Y_neg[mask_neg]
X_neg = X_neg[mask_neg]

X_plt = np.concatenate((X_pos,X_neg))
Y_plt = np.concatenate((Y_pos,Y_neg))

bins = 75
plt.hist2d(X_plt,Y_plt, bins = bins, density= True, cmap = "gist_ncar")
plt.colorbar()

plt.xlabel("x coordinate of the prior", fontsize = 12)
plt.ylabel("y coordinate of the prior", fontsize = 12)